In [2]:
import json 
with open('LLM.CureDoc_Evaluation.json', 'r') as file:
    old_format = json.load(file)

def transform_entry(entry):
    """
    Transform an old format entry to the new format.
    """
    transformed = {
        "input": entry["input"],
        "output": entry["output"],
        "Doc1": {
            "Cd_rating": {"1st": "", "2nd": "", "3rd": "", "4th": "", "suggestion": ""},
            "gpt_rating": {"1st": "", "2nd": "", "3rd": "", "4th": "", "suggestion": ""}
        },
        "Doc2": {
            "Cd_rating": {"1st": "", "2nd": "", "3rd": "", "4th": entry.get('rating',0), "suggestion": entry.get("suggestions", "")},
            "gpt_rating": {"1st": "", "2nd": "", "3rd": "", "4th": "", "suggestion": ""}
        },
        "Doc3": {
            "Cd_rating": {"1st": "", "2nd": "", "3rd": "", "4th": "", "suggestion": ""},
            "gpt_rating": {"1st": "", "2nd": "", "3rd": "", "4th": "", "suggestion": ""}
        },
        "gpt_output": entry["gpt_output"]
    }
    return transformed

def convert_format(old_format):
    """
    Convert the entire old format to the new one.
    """
    new_format = []
    for old_data in old_format:
        new_data = {
            "_id": old_data["_id"],
            "Name": old_data["Name"],
            "entries": [transform_entry(entry) for entry in old_data["entries"]]
        }
        new_format.append(new_data)
    return new_format

new_format = convert_format(old_format)

with open('LLM.CureDoc_Evaluation_test.json', 'w') as file:
    json.dump(new_format, file, indent=4)


In [7]:
from pymongo import MongoClient
import json

# MongoDB connection setup
client = MongoClient('mongodb://172.16.101.171:27017/')
db = client['LLM']  # Database name
collection = db['CureDoc_Evaluation']  # Collection name

# Fetch all documents from the collection
data = list(collection.find({}))

# Convert the MongoDB documents to a format that can be JSON serialized.
# ObjectIds are not serializable by default, so we convert them to strings.
for doc in data:
    doc['_id'] = str(doc['_id'])

# Save the data to a JSON file
with open('CureDoc_Evaluation_data_1.json', 'w') as f:
    json.dump(data, f, indent=4)

# Close the MongoDB connection
client.close()

print("Data downloaded and saved to CureDoc_Evaluation_data.json")


Data downloaded and saved to CureDoc_Evaluation_data.json


In [3]:
import json
from pymongo import MongoClient

In [17]:
import json

# Function to replace keys in both Cd_rating and gpt_rating
def replace_keys_in_ratings(data):
    for doc in data:
        if "entries" in doc:  # Ensure 'entries' key exists
            for entry in doc["entries"]:
                # Process each doctor's ratings
                for doc_key in ['Doc1', 'Doc2', 'Doc3']:
                    doctor = entry.get(doc_key)
                    if doctor is not None:  # Check if the doctor key exists in the entry
                        # Apply changes to both 'Cd_rating' and 'gpt_rating' for each doctor
                        for rating_key in ["Cd_rating", "gpt_rating"]:
                            rating = doctor.get(rating_key)
                            if rating is not None:  # Ensure the rating exists and is a dictionary
                                # Only add new keys if they don't already exist
                                if "1st" in rating and "Clarity" not in rating:
                                    rating["Clarity"] = rating.pop("1st")
                                if "2nd" in rating and "Accuracy" not in rating:
                                    rating["Accuracy"] = rating.pop("2nd")
                                if "3rd" in rating and "Brevity" not in rating:
                                    rating["Brevity"] = rating.pop("3rd")
                                if "4th" in rating and "Relevance" not in rating:
                                    rating["Relevance"] = rating.pop("4th")

# Read data from input.json
with open('CureDoc_Evaluation_data_1.json', 'r') as f:
    data = json.load(f)

# Apply the modifications
replace_keys_in_ratings(data)

# Save the modified data into output.json
with open('CureDoc_Evaluation_test_final_1.json', 'w') as f:
    json.dump(data, f, indent=4)

print("Data has been successfully modified and saved to output.json")


Data has been successfully modified and saved to output.json


In [6]:
from pymongo import MongoClient

# Function to replace keys in both Cd_rating and gpt_rating
def replace_keys_in_ratings(entry):
    for key in ["Cd_rating", "gpt_rating"]:
        for doc_key, doc_rating in entry.items():
            if key in doc_rating:
                rating = doc_rating[key]
                rating["Clarity"] = rating.pop("1st", None)
                rating["Accuracy"] = rating.pop("2nd", None)
                rating["Brevity"] = rating.pop("3rd", None)
                rating["Relevance"] = rating.pop("4th", None)

# MongoDB connection setup
client = MongoClient('mongodb://172.16.101.171:27017/')
db = client['LLM']  # Replace with your database name
collection = db['CureDoc_Evaluation']  # Replace with your collection name

# Fetch data from MongoDB
data = collection.find({})

# Apply the modifications to each document
for doc in data:
    for entry in doc["entries"]:
        replace_keys_in_ratings(entry)

    # Update the document in MongoDB
    # collection.update_one({'_id': doc['_id']}, {'$set': doc})

# Close the MongoDB connection
client.close()

TypeError: argument of type 'int' is not iterable